In [1]:
import pandas as pd
import numpy as np

In [13]:
#Read in the datafile (data grabbed from API using 'get_data.py')
df_cityData = pd.read_csv('2017_NYC_Data.csv', low_memory=False)
print("Finished! Data now available.")

Finished! Data now available.


##Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

filter out the top 10 complaint types

In [14]:
df_topTenComplaintTypes = df_cityData['complaint_type'].value_counts().head(10)
df_topTenComplaintTypes

Noise - Residential        230355
HEAT/HOT WATER             213630
Illegal Parking            146213
Blocked Driveway           136156
Street Condition            93336
Street Light Condition      84298
UNSANITARY CONDITION        79349
Noise - Street/Sidewalk     73119
Water System                65155
Noise                       60217
Name: complaint_type, dtype: int64

filter the city data further using the above results

In [15]:
#filter the records for which the complaint type matches one found in 'df_topTenComplaintTypes'
df_TopTenData = df_cityData.loc[df_cityData['complaint_type'].isin(df_topTenComplaintTypes.index)]
#filter the records to include only the 'borough' & 'complaint_type' (multiple for each borough), and the number of occurences for each complaint within that borough
df_TopTenData[['borough','complaint_type']].groupby(['borough','complaint_type'])['complaint_type'].count()

borough        complaint_type         
BRONX          Blocked Driveway           24601
               HEAT/HOT WATER             68744
               Illegal Parking            16139
               Noise                       3135
               Noise - Residential        57736
               Noise - Street/Sidewalk    14032
               Street Condition           11774
               Street Light Condition     18412
               UNSANITARY CONDITION       24600
               Water System               10236
BROOKLYN       Blocked Driveway           49409
               HEAT/HOT WATER             67026
               Illegal Parking            55501
               Noise                      15426
               Noise - Residential        67714
               Noise - Street/Sidewalk    21329
               Street Condition           25449
               Street Light Condition     22497
               UNSANITARY CONDITION       26662
               Water System               19819
M

##Consider only the 10 most common overall complaint types.  For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?

read in the new dataset and merge the population data with the city data

In [16]:
#read in the dataset containing the population data
df_populationData = pd.read_csv('PopulationData.csv')

In [17]:
#merge the two dataframes
df_populationData.columns = ['incident_zip','population'] #rename columns to match other df
df_populationData['incident_zip'] = df_populationData['incident_zip'].apply(str)#change column type to string to handle 'NaN' in df_zipComplaintType
df_zipComplaintType = df_cityData[['incident_zip','complaint_type']] #grab necessary data from df_cityData
df_mergedData = pd.merge(df_populationData, df_zipComplaintType, on='incident_zip')

apply filtering and grab the necessary data

In [18]:
df_overlappingZips = df_mergedData.loc[df_mergedData['incident_zip'].isin(df_cityData['incident_zip'])] #filter for overlapping zip codes
df_overlappingZips = df_overlappingZips.loc[df_overlappingZips['complaint_type'].isin(df_topTenComplaintTypes.index)] #filter for top ten complaints
df_overlappingZips = df_overlappingZips.groupby(['incident_zip','population','complaint_type'])['complaint_type'].count()
df_overlappingZips = df_overlappingZips.to_frame('').sort_values(by='population', ascending=False).head(100) #sort and grab first 100 (10zips x 10complaints = 100rows)
print(df_overlappingZips.to_string()) #print ALL data rows 

                                                     
incident_zip population complaint_type               
11368        109931     UNSANITARY CONDITION      641
                        Street Light Condition    447
                        Blocked Driveway         4383
                        HEAT/HOT WATER           1620
                        Illegal Parking          1252
                        Noise                     158
                        Noise - Residential      2462
                        Street Condition          561
                        Noise - Street/Sidewalk   684
                        Water System              615
11226        101572     Noise - Residential      4856
                        Blocked Driveway         2204
                        Water System              407
                        Illegal Parking          1077
                        Noise                     440
                        HEAT/HOT WATER           7574
                        Nois

##Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017? Meaning, calculate a complaint-index that adjusts for population of the borough.

grab the total # of complaints for each borough

In [19]:
df_complaintCountPerBorough = df_cityData[['borough','complaint_type']].groupby(['borough'])['complaint_type'].count().reset_index()
df_complaintCountPerBorough.rename(columns = {'complaint_type': 'complaint_count'}, inplace=True)
df_complaintCountPerBorough

,borough,complaint_count
0,BRONX,451120
1,BROOKLYN,771157
2,MANHATTAN,480508
3,QUEENS,590767
4,STATEN ISLAND,129512


grab the total population of each borough

In [20]:
#Grab necessary data from df_cityData
df_zipBorough = df_cityData[['borough','incident_zip']]
df_zipBorough = df_zipBorough.replace(['0', 'N/A', 'UNKNOWN', 'NA','.',''], pd.np.nan).dropna().drop_duplicates() #filter out 'bad' rows

#Grab necessary data from df_populationData
df_zipPop = df_populationData[['incident_zip','population']]

#Merge and manipulate data to get the total population value for each borough
df_populationOfBoroughs = pd.merge(df_zipBorough, df_zipPop, on='incident_zip')
df_populationOfBoroughs = df_populationOfBoroughs.sort_values(by='borough')
df_populationOfBoroughs = df_populationOfBoroughs.groupby(['borough'])['population'].sum().reset_index()
df_populationOfBoroughs

,borough,population
0,BRONX,1592084
1,BROOKLYN,2732303
2,MANHATTAN,2073659
3,QUEENS,2755424
4,STATEN ISLAND,468730


merge the two tables shown above

In [21]:
df_biggestComplainers = pd.merge(df_complaintCountPerBorough, df_populationOfBoroughs, on='borough')
#calculate complaint index (complaint_count/population) for each borough
df_biggestComplainers['complaint_index'] = df_biggestComplainers['complaint_count']/df_biggestComplainers['population']
df_biggestComplainers.sort_values(by='complaint_index', ascending=False) #sort by biggest "complainers"

,borough,complaint_count,population,complaint_index
0,BRONX,451120,1592084,0.283352
1,BROOKLYN,771157,2732303,0.282237
4,STATEN ISLAND,129512,468730,0.276304
2,MANHATTAN,480508,2073659,0.231720
3,QUEENS,590767,2755424,0.214401


RESULT: Based on the above data, we can easily see that people from the Bronx are the biggest "complainers" with an average of around 1 complaint per 3.5 people. 